<a href="https://colab.research.google.com/github/arunshaju001/Crime-Prediction-using-Machine-Learning/blob/master/Project_crime.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Crime Prediction and Visualization**

## **Libraries**

In [1]:
#Libraries for map 
!apt-get install libgeos-dev
!pip install https://github.com/matplotlib/basemap/archive/master.zip
from mpl_toolkits.basemap import Basemap

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Suggested packages:
  libgdal-doc
The following NEW packages will be installed:
  libgeos-dev
0 upgraded, 1 newly installed, 0 to remove and 25 not upgraded.
Need to get 73.1 kB of archives.
After this operation, 486 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libgeos-dev amd64 3.6.2-1build2 [73.1 kB]
Fetched 73.1 kB in 0s (164 kB/s)
Selecting previously unselected package libgeos-dev.
(Reading database ... 134448 files and directories currently installed.)
Preparing to unpack .../libgeos-dev_3.6.2-1build2_amd64.deb ...
Unpacking libgeos-dev (3.6.2-1build2) ...
Setting up libgeos-dev (3.6.2-1build2) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
     \ 213.1MB 452kB/s
     |████████████████████████████████| 10.4MB 3.2MB/s 
     |████████████████████████████████| 225kB 63.3MB/s 
  Created wheel for basemap: filename=basemap

In [0]:
import pandas as pd
import numpy as np
import datetime
import time
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB 
from sklearn import metrics 
import json
import matplotlib as mpl
import folium
from folium.plugins import MarkerCluster
import math
import matplotlib.pyplot as plt
%matplotlib inline

## **Mapping Lattitudes and Longitudes to Grids** - No need to Run!!!

In [0]:
#Creating dataset with grid value
# df.to_csv('/content/drive/My Drive/Colab Notebooks/Project/grid_data_large.csv', index = False)

In [0]:
#Rmoving rows with offset values
# df=df[df.Y <= 37.811127]
# df=df[ df.Y>=37.708318]
# df=df[df.X <= -122.356992]
# df=df[ df.X>=-122.514985]

In [0]:
#Lat Long Mapping
#Dont run!!!

# def get_grid_val(lat_x,lon_y):
#   lat_x=37.811128-lat_x
#   lat_x=lat_x/.0102809
#   lat_x=math.floor(lat_x)
#   lon_y=-122.514985-lon_y
#   lon_y=lon_y/.0157993
#   lon_y=abs(lon_y)
#   lon_y=math.floor(lon_y)
#   xy=lon_y*10+lat_x
#   return(xy)

# d=[]
# for index, row in df.iterrows(): 
#     d.append(get_grid_val(row['Y'], row['X']))
# df["grid"]=d
# print("success")

## **Preparing Dataset for Training**

**Read dataset from drive and write into Local**

In [0]:
#Read dataset from drive and write into sample_data
#sfnew - orginal
#grid_data - small with grid
#train - small ds
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Project/grid_data_large.csv')
#Saving to local
data.to_csv('/content/sample_data/sfnew.csv', index=False)
#Reading the Dataset
df=pd.read_csv('/content/sample_data/sfnew.csv')

**Creating year, month, day and hour columns**

In [0]:
#Creating year, month and day columns
df['year'] = pd.DatetimeIndex(df['Date']).year
df['month'] = pd.DatetimeIndex(df['Date']).month
df['day'] = pd.DatetimeIndex(df['Date']).day
# df['hour'] = pd.DatetimeIndex(df['Date']).hour  #In crime,grid_data
df['hour'] = pd.DatetimeIndex(df['Time']).hour    #In sfnew,grid_data_large

**Category conversion**

In [0]:
#Category conversion
crime_dict={  'WARRANTS':1,   #c7000a
              'LARCENY/THEFT':2, 'VEHICLE THEFT':2,'ROBBERY':2,'STOLEN PROPERTY':2,'EXTORTION':2, 'EMBEZZLEMENT':2,   #ed0274
              'VANDALISM':3,'ARSON':3,    #a6039e
              'ASSAULT':4, #830fa6
              'WEAPON LAWS':5,  #6b09eb
              'BURGLARY':6,'TRESPASS':6,    #171ee3          
              'MISSING PERSON':7,  'KIDNAPPING':7,    #2e86f2
              'SEX OFFENSES NON FORCIBLE':8, 'PROSTITUTION':8,'SEX OFFENSES, NON FORCIBLE':8,   #00b6c7
              'SEX OFFENSES FORCIBLE':9, 'SEX OFFENSES, FORCIBLE':9,   #00c25e            
              'LOITERING':10, 'SUICIDE':10,'DISORDERLY CONDUCT':10, 'DRIVING UNDER THE INFLUENCE':10, #none           
              'TREA':10, 'RECOVERED VEHICLE':10, 'OTHER OFFENSES':10,'BAD CHECKS':10,'GAMBLING':10,
              'FORGERY/COUNTERFEITING':10,'SECONDARY CODES':10,'NON-CRIMINAL':10,'SUSPICIOUS OCC':10, 'DRUG/NARCOTIC':10,  
              'FRAUD':10, 'RUNAWAY':10,'DRUNKENNESS':10, 'FAMILY OFFENSES':10,'LIQUOR LAWS':10,  'BRIBERY':10,        
              'PORNOGRAPHY/OBSCENE MAT':11  #c27e00 
              }
df["Category"] = df["Category"].replace(crime_dict)

**Day of weeks conversion**

In [0]:
#Day of weeks conversion
data_week_dict = { "Monday": 1, "Tuesday":2, "Wednesday":3, "Thursday":4, "Friday":5, "Saturday":6, "Sunday":7 }
df["DayOfWeek"] = df["DayOfWeek"].replace(data_week_dict)

 **District conversion**

In [0]:
#District conversion
district = df["PdDistrict"].unique()
data_dict_district = {}
count = 1
for data in district:
    data_dict_district[data] = count
    count+=1 
df["PdDistrict"] = df["PdDistrict"].replace(data_dict_district)

**Checking Data**

In [11]:
#Checking correlation
corr = df.corr()
print(corr["Category"])

IncidntNum   -0.039038
Category      1.000000
DayOfWeek    -0.026793
PdDistrict    0.038569
X             0.006231
Y            -0.046726
PdId         -0.039038
grid          0.017282
year         -0.031345
month        -0.014189
day          -0.020276
hour         -0.071228
Name: Category, dtype: float64


In [12]:
#Checking data distribution
skew = df.skew()
print(skew)

IncidntNum    0.705717
Category      0.078545
DayOfWeek    -0.008013
PdDistrict    0.093789
X            -1.208264
Y            -0.725909
PdId          0.705717
grid         -1.040506
year         -0.030696
month         0.021918
day           0.008271
hour         -0.516175
dtype: float64


## **Feature Selection and Creation of Prediction Dataset**

**Selecting features and Splitting Dataset**

In [0]:
#Selecting features
# "DayOfWeek" "PdDistrict" "X" "Y" "split" "grid" "year" "month" "day" "hour"
features = [  "grid","year","DayOfWeek" ,"month","PdDistrict","day" ]
df_new=df[features]
df_res=df.Category
#Splitting the data
train_X, test_X, train_y, test_y = train_test_split(df_new, df_res, train_size=0.9 , random_state = 1)

**Creating prediction_set for Predicting Crime**

In [0]:
# Creating prediction_set for Predicting Crime
columns=[ "grid","year","DayOfWeek" ,"month","PdDistrict","day" ]
pd.DataFrame(index=np.arange(1), columns=np.arange(8))
prediction_set = pd.DataFrame( columns=columns)
prediction_set = prediction_set.fillna(0)

#District assignment to Grid
grid_Pd_dict ={ 2: 6, 3: 6, 4: 2, 5: 2, 6: 2, 7: 2, 8: 2, 12: 6, 13: 6, 14: 2, 15: 2, 16: 2, 17: 2, 18: 2, 19: 2, 20: 4,
                22: 6, 23: 6, 24: 2, 25: 2, 26: 2, 27: 2, 28: 2, 29: 2, 30: 8, 31: 6, 32: 6, 33: 10, 34: 2, 35: 10,
                36: 2, 37: 2, 38: 2, 39: 2, 40: 4, 41: 4, 42: 6, 43: 10, 44: 10, 45: 10, 46: 5, 47: 1, 48: 2,
                49: 2, 50: 4, 51: 4, 52: 4, 53: 4, 54: 10, 55: 7, 56: 5, 57: 5, 58: 5, 59: 5, 60: 8, 61: 8, 62: 8,
                63: 1, 64: 7, 65: 7, 66: 3, 67: 3, 68: 3, 69: 5, 70: 8, 71: 1, 72: 8, 73: 1, 74: 3, 75: 3, 76: 3,
                77: 3, 78: 3, 79: 3, 82: 1, 83: 3, 84: 3, 85: 3, 86: 3, 87: 3, 88: 3, 89: 3, 90: 1, 97: 3, 98: 3}

#Grid Correction
grid_val={    0 : 90, 10 : 91, 20 : 92, 30 : 93, 40 : 94, 50 : 95, 60 : 96, 70 : 97, 80 : 98, 90 : 99,
              1 : 80, 11 : 81, 21 : 82, 31 : 83, 41 : 84, 51 : 85, 61 : 86, 71 : 87, 81 : 88, 91 : 89,
              2 : 70, 12 : 71, 22 : 72, 32 : 73, 42 : 74, 52 : 75, 62 : 76, 72 : 77, 82 : 78, 92 : 79,
              3 : 60, 13 : 61, 23 : 62, 33 : 63, 43 : 64, 53 : 65, 63 : 66, 73 : 67, 83 : 68, 93 : 69,
              4 : 50, 14 : 51, 24 : 52, 34 : 53, 44 : 54, 54 : 55, 64 : 56, 74 : 57, 84 : 58, 94 : 59,
              5 : 40, 15 : 41, 25 : 42, 35 : 43, 45 : 44, 55 : 45, 65 : 46, 75 : 47, 85 : 48, 95 : 49,
              6 : 30, 16 : 31, 26 : 32, 36 : 33, 46 : 34, 56 : 35, 66 : 36, 76 : 37, 86 : 38, 96 : 39,
              7 : 20, 17 : 21, 27 : 22, 37 : 23, 47 : 24, 57 : 25, 67 : 26, 77 : 27, 87 : 28, 97 : 29,
              8 : 10, 18 : 11, 28 : 12, 38 : 13, 48 : 14, 58 : 15, 68 : 16, 78 : 17, 88 : 18, 98 : 19,
              9 : 0, 19 : 1, 29 : 2, 39 : 3, 49 : 4, 59 : 5, 69 : 6, 79 : 7, 89 : 8, 99 : 9}

#Adding Data to prediction_set 
grid_set=[]
for i in df.grid.unique():
  grid_set.append(i)
prediction_set["grid"]=grid_set
prediction_set["year"]=2020
prediction_set["DayOfWeek"]=5
prediction_set["month"]=3
prediction_set["day"]=6
prediction_set["PdDistrict"]=df["grid"].replace(grid_Pd_dict)

**Grid to District Mapping** - No need to Run!!!

In [0]:
#Related Works
#Dont run!!!
# col_one_list = districtToGrid['grid'].tolist()
# col_two_list = districtToGrid['PdDistrict'].tolist()

# for i in range(len(col_one_list)):
#   d[col_one_list[i]]=col_two_list[i]
# d
# districtToGrid

## **Model Training and Prediction Using KNN**

**KNN Classifier**

In [16]:
#KNN 
#2496 cuda cores-gpu
knn = KNeighborsClassifier(n_neighbors=5,weights='distance',n_jobs=2)
knn.fit(train_X, train_y)
print("Success")

Success


**Predicted Values and Preparing values for Map**

In [0]:
#Predicted Values and Preparing values for Map
predicted_category = knn.predict(prediction_set[features])
grid_list = prediction_set["grid"].tolist()

map_val_dict={}
for i in range(len(grid_list)):
  map_val_dict[grid_list[i]]=predicted_category[i]

#Adding missing grids in map_val_dict
miss_grid=[0,1,9,10,11,21,80,81,91,92,93,94,95,96,99]
for i in miss_grid:
  map_val_dict[i]=10
# map_val_dict

**KNN Accuracy on Test Dataset**

In [0]:
#KNN Accuracy
# predictions_knn = knn.predict(test_X)
# print("KNN Accuracy :", metrics.accuracy_score(test_y, predictions_knn)*100)

# KNN Accuracy : 41.51674429867669 n=5,size=.9, "PdDistrict", "X" ,"hour", "grid"   #test
# KNN Accuracy : 41.95911914172783 n=5,size=.9, "PdDistrict", "X" ,"hour", "grid"   #orginal

## **Other Models** - No need to Run!!!

In [0]:
#Logistic Regression
  # log = LogisticRegression(max_iter=4000,n_jobs=2)
  # log.fit(train_X, train_y)
  # predictions_lr = log.predict(test_X)
  # print("Success")

In [0]:
#Logistic Regression Accuracy
  # print("Gaussian Naive Bayes Accuracy :", metrics.accuracy_score(test_y, predictions_lr)*100)

In [0]:
#Gaussian Naive Bayes
  # gnb = GaussianNB() 
  # gnb.fit(train_X, train_y) 
  # predictions_gnb = gnb.predict(test_X) 
  # print("Success")

In [0]:
#Gaussian Naive Bayes Accuracy
  # print("Gaussian Naive Bayes Accuracy :", metrics.accuracy_score(test_y, predictions_gnb)*100)

## **Map Visualization**

### Grid Function for Map

In [0]:
#Grid Function
def get_geojson_grid(upper_right, lower_left, n=10):

    all_boxes = []

    lat_steps = np.linspace(lower_left[0], upper_right[0], n+1)
    lon_steps = np.linspace(lower_left[1], upper_right[1], n+1)

    lat_stride = lat_steps[1] - lat_steps[0]
    lon_stride = lon_steps[1] - lon_steps[0]
    grid_index=0
    for lat in lat_steps[:-1]:
        for lon in lon_steps[:-1]:
            # Define dimensions of box in grid
            upper_left = [lon, lat + lat_stride]
            upper_right = [lon + lon_stride, lat + lat_stride]
            lower_right = [lon + lon_stride, lat]
            lower_left = [lon, lat]

            # Define json coordinates for polygon
            coordinates = [
                upper_left,
                upper_right,
                lower_right,
                lower_left,
                upper_left
            ]

            geo_json = {"type": "FeatureCollection",
                        "properties":{
                            "lower_left": lower_left,
                            "upper_right": upper_right,
                            "name": grid_index,
                            "category": int(map_val_dict[grid_index]),
                            
                        },
                        "features":[]}

            grid_feature = {
                "type":"Feature",
                "geometry":{
                    "type":"Polygon",
                    "coordinates": [coordinates]
                    
                }
            }
            grid_index+=1
            geo_json["features"].append(grid_feature)

            all_boxes.append(geo_json)
    
    return all_boxes

### **Map Visualization**



In [33]:
#Map with Grid
#End Coordinates
lower_left = [37.708318, -122.514985]
upper_right = [37.811127, -122.356992]

SF_COORDINATES = (37.76, -122.45)
m = folium.Map(location=SF_COORDINATES, zoom_start=12)  #Creating Map
grid = get_geojson_grid(upper_right, lower_left , n=10) #Grid Creation
# style_function = lambda x: {'fillColor': '#0000ff' if (x['properties']['name'])%2 == 0 else '#00ff00'}

for i, geo_json in enumerate(grid):


    # color = plt.cm.Reds(i / len(grid))
    # color = mpl.colors.to_hex(color)
    blank_list=[49,59,69,79,89,99,98,88,80,90,91,81]  #Empty cells
    if(i in blank_list): #or geo_json['properties']['category']==10):    #For No Crime
      continue
    elif(geo_json['properties']['category']==1):    #WARRANTS
      gj1 = folium.GeoJson(geo_json,
                          style_function=lambda feature : {'fillColor': "#c7000a"
                          ,'color':"black"
                          , 'opacity':0.35,'weight': 1,'fillOpacity': 0.65,} 
                          ).add_to(m) 
      m.add_child(gj1)
    elif(geo_json['properties']['category'] == 2):    #THEFT
      gj = folium.GeoJson(geo_json,
                          style_function=lambda feature : {'fillColor': "#ed0274"   #pink
                          ,'color':"black"
                          , 'opacity':0.35,'weight': 1,'fillOpacity': 0.65,} 
                          ).add_to(m) 
      m.add_child(gj)
    elif(geo_json['properties']['category'] == 3):    #ARSON
        gj = folium.GeoJson(geo_json,
                            style_function=lambda feature : {'fillColor': "#dff702"
                            ,'color':"black"
                            , 'opacity':0.35,'weight': 1,'fillOpacity': .65,} 
                            ).add_to(m) 
        m.add_child(gj)
    elif(geo_json['properties']['category'] == 4):    #ASSAULT
      gj = folium.GeoJson(geo_json,
                          style_function=lambda feature : {'fillColor': "#830fa6"
                          ,'color':"black"
                          , 'opacity':0.35,'weight': 1,'fillOpacity': 0.65,} 
                          ).add_to(m) 
      m.add_child(gj)
    elif(geo_json['properties']['category']== 5):   #WEAPON LAWS
      gj = folium.GeoJson(geo_json,
                          style_function=lambda feature : {'fillColor': "#6b09eb"
                          ,'color':"black"
                          , 'opacity':0.35,'weight': 1,'fillOpacity': 0.65,} 
                          ).add_to(m) 
      m.add_child(gj)
    elif(geo_json['properties']['category'] == 6):  #BURGLARY
      gj = folium.GeoJson(geo_json,
                          style_function=lambda feature : {'fillColor': "#171ee3 "
                          ,'color':"black"
                          , 'opacity':0.35,'weight': 1,'fillOpacity': 0.65,} 
                          ).add_to(m) 
      m.add_child(gj)
    elif(geo_json['properties']['category'] == 7):    #KIDNAPPING
      gj = folium.GeoJson(geo_json,
                          style_function=lambda feature : {'fillColor': "#2e86f2"
                          ,'color':"black"
                          , 'opacity':0.35,'weight': 1,'fillOpacity': 0.65,} 
                          ).add_to(m) 
      m.add_child(gj)
    elif(geo_json['properties']['category'] == 8):    #SEX OFFENSES
      gj = folium.GeoJson(geo_json,
                          style_function=lambda feature : {'fillColor': "#00b6c7"
                          ,'color':"black"
                          , 'opacity':0.35,'weight': 1,'fillOpacity': 0.65,} 
                          ).add_to(m) 
      m.add_child(gj)
    elif(geo_json['properties']['category'] == 9):    #SEX OFFENSES FORCIBLE
      gj = folium.GeoJson(geo_json,
                          style_function=lambda feature : {'fillColor': "#00c25e"
                          ,'color':"black"
                          , 'opacity':0.35,'weight': 1,'fillOpacity': 0.65,} 
                          ).add_to(m) 
      m.add_child(gj)
    elif(geo_json['properties']['category'] == 11):   #OBSCENE
      gj = folium.GeoJson(geo_json,
                          style_function=lambda feature : {'fillColor': "#c27e00 "
                          ,'color':"black"
                          , 'opacity':0.35,'weight': 1,'fillOpacity': 0.65,} 
                          ).add_to(m) 
      m.add_child(gj)
    else:
      gj2=folium.GeoJson(geo_json,
                          style_function=lambda feature : {'fillColor': "black" 
                          ,'color':"black"
                          , 'opacity':0,'weight': 1,'fillOpacity': 0.05,} 
                          )
      m.add_child(gj2) 
    # folium.Marker([37.775421		,-122.403405	], popup= "here" ,  icon=folium.Icon(color='red')).add_to(m)

m
# import IPython
# IPython.display.HTML(filename="/content/out.html")